In [1]:
import pandas as pd
import os
import glob
com = pd.concat(map(pd.read_csv, glob.glob('*.csv')))
ttc = com[['RatID','Stage','Correct/Incorrect']].groupby(['RatID','Stage']).sum()
ttc.rename(columns={'Correct/Incorrect':'trials to criterion'},inplace=True)
def getttc(id, stage):
    return ttc.loc[id, stage]

getttc(39, 'ED')

trials to criterion    14
Name: (39, ED), dtype: int64